In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
tf.__version__

'1.1.0'

In [9]:
class FFCell:
    def __init__(self, input_size, output_size, hidden_desc, regression = False):
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_desc = hidden_desc
        self.regression = regression
        self.layers = []
        self.ops = []
        self.X_train = tf.placeholder(tf.float32, [None, input_size], name="X_train")
        self.y_train = tf.placeholder(tf.float32, name="y_train")
        self._init_weights()
        self._build_graph()
    
    def _init_weights(self):
        self.layers.append({ 'weights': tf.Variable(tf.random_normal([self.input_size, self.hidden_desc[0]['hidden_nodes']])),
                             'biases': tf.Variable(tf.random_normal([self.hidden_desc[0]['hidden_nodes']]))})
        
        if len(self.hidden_desc[1:]) >= 1:
            cnt = 0
            for desc in self.hidden_desc[1:]:
                i_size = self.hidden_desc[cnt]['hidden_nodes']
                self.layers.append({ 'weights': tf.Variable(tf.random_normal([i_size, desc['hidden_nodes']])),
                                     'biases': tf.Variable(tf.random_normal([desc['hidden_nodes']]))})
                cnt += 1
        
        self.layers.append({ 'weights': tf.Variable(tf.random_normal([self.hidden_desc[len(self.hidden_desc) - 1]['hidden_nodes'], self.output_size])),
                             'biases': tf.Variable(tf.random_normal([self.output_size]))})
    
    def _build_graph(self):
        activ = self.hidden_desc[0]['activation']
        self.ops.append(
            activ(tf.matmul(self.X_train, self.layers[0]['weights']) + self.layers[0]['biases'])
        )
        
        if len(self.layers[1:-1]) >= 1:
            cnt = 1
            for layer in self.layers[1:-1]:
                activ = self.hidden_desc[cnt]['activation']
                self.ops.append(
                    activ(tf.matmul(self.layers[cnt-1]['weights'], layer['weights']) + layer['biases'])
                )
                cnt += 1
        
        self.ops.append(
                tf.matmul(self.layers[len(self.layers) - 2]['weights'], self.layers[len(self.layers) - 1]['weights']) +
                self.layers[len(self.layers) - 1]['biases']
        )
    
    def _get_next_batch(x, y, epoch, batch_size):
        start = epoch * batch_size
        end = start + batch_size
        
        if end >= len(x):
            end = len(x)
        
        return (x[start:end], y[start:end])
    
    def predict(self):
        return 0
    
    def train(self, mnist, batch_size, epochs):
        # TODO do train stuff
        pred = self.ops[len(self.ops) - 1]
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=self.y_train))
        
        optimizer = tf.train.AdamOptimizer().minimize(cost)
        
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for epoch in range(epochs):
                epoch_cost = 0
                for _ in range(int(mnist.train.num_examples/batch_size)):
                    x, y = mnist.train.next_batch(batch_size)
                    _, c = sess.run([optimizer, cost], feed_dict = {self.X_train: x, self.y_train: y})
                    epoch_cost += c
                print('Epoch: ', (epoch + 1), '/', epochs, ' Epoch cost: ', epoch_cost)
#             correct = tf.equal(tf.argmax(pred, 1), tf.argmax(self.y_train, 1))
#             accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
#             print('Accuracy', accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))
    

In [11]:
mnist = input_data.read_data_sets('../Data/MNIST/', one_hot=True)

hidden_desc = [
    {'hidden_nodes': 500, 'activation': tf.nn.relu},
    {'hidden_nodes': 500, 'activation': tf.nn.relu},
    {'hidden_nodes': 500, 'activation': tf.nn.relu}
]

ffcell = FFCell(784, 10, hidden_desc, True)

print(ffcell.layers)
print(ffcell.ops)

# ffcell.train(mnist, 100, 10)


Extracting ../Data/MNIST/train-images-idx3-ubyte.gz
Extracting ../Data/MNIST/train-labels-idx1-ubyte.gz
Extracting ../Data/MNIST/t10k-images-idx3-ubyte.gz
Extracting ../Data/MNIST/t10k-labels-idx1-ubyte.gz
[{'weights': <tf.Variable 'Variable_32:0' shape=(784, 500) dtype=float32_ref>, 'biases': <tf.Variable 'Variable_33:0' shape=(500,) dtype=float32_ref>}, {'weights': <tf.Variable 'Variable_34:0' shape=(500, 500) dtype=float32_ref>, 'biases': <tf.Variable 'Variable_35:0' shape=(500,) dtype=float32_ref>}, {'weights': <tf.Variable 'Variable_36:0' shape=(500, 500) dtype=float32_ref>, 'biases': <tf.Variable 'Variable_37:0' shape=(500,) dtype=float32_ref>}, {'weights': <tf.Variable 'Variable_38:0' shape=(500, 10) dtype=float32_ref>, 'biases': <tf.Variable 'Variable_39:0' shape=(10,) dtype=float32_ref>}]
[<tf.Tensor 'Relu_12:0' shape=(?, 500) dtype=float32>, <tf.Tensor 'Relu_13:0' shape=(784, 500) dtype=float32>, <tf.Tensor 'Relu_14:0' shape=(500, 500) dtype=float32>, <tf.Tensor 'add_15:0' sh